In [ ]:
import backtrader as bt
from backtester import backtest15,backtestday,backtest1hr
import numpy as np
import statistics            
import pandas as pd
import pyfolio
import datetime
from datetime import datetime
import math
import logging
logging.basicConfig(filename="log.txt", level=logging.DEBUG)

In [ ]:
!conda uninstall theano

In [ ]:
!pip install quantstats
import quantstats 

In [133]:
pnl = np.array([])

In [134]:
# pnl=np.array([])
class Nimisha(bt.Strategy):
    
    def __init__(self):
        self.lookback=15
        
    def log(self, txt):
        date = self.data.datetime.date(0)
        time = self.data.datetime.time()
        logging.debug('%s-%s, %s' % (date.isoformat(), time, txt))

#     def get_pnl(self):
#         return self.pnl

    def notify_order(self, order):

        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log("Buy EXECUTED {}".format(order.executed.price))
            elif order.issell():
                self.log("Sell EXECUTED {}".format(order.executed.price))

    def next(self):
        ohlc1 = (self.data.close[-1]+self.data.high[-1]+self.data.low[-1]+self.data.open[-1])/4
        ohlc2 = (self.data.close[-2]+self.data.high[-2]+self.data.low[-2]+self.data.open[-2])/4
        ohlc3 = (self.data.close[-3]+self.data.high[-3]+self.data.low[-3]+self.data.open[-3])/4
        ohlc = self.data.close.get(size=self.lookback,ago=-1)+self.data.high.get(size=self.lookback,ago=-1)+self.data.low.get(size=self.lookback,ago=-1)+self.data.open.get(size=self.lookback,ago=-1)
        ohlc = np.array(ohlc)
        ohlc/=4
        std = np.array(ohlc).std()
        cond1 = ohlc1>ohlc2 and ohlc2>ohlc3
        cond2 = (ohlc1>ohlc2+std/2)
        cond3 = self.data.high[-1]<self.data.close[0]
        lcond = (cond1 or cond2) and cond3
        if(lcond):
            if(self.data.high[0]>self.data.open[0]*1.005):
                pnl = np.append(pnl,self.data.open[0]*0.005)
            else:
                pnl = np.append(pnl,self.data.close[0]-self.data.open[0])

In [136]:
def analysis(data):
    pnl=np.array([])
    nim = Nimisha()
    try:
        backtest1hr(data, nim)
    except:
        pass
#     pnl = np.array(pnl)

#     aa = nim.get_pnl()
    aa = pnl
    return aa.sum(), len(aa),len(aa[aa>0]),len(aa[aa<0])

In [137]:
analysis('RELIANCE')

Starting Portfolio Value: 100000.00


(0.0, 0, 0, 0)

IndentationError: expected an indented block (1763208468.py, line 3)

In [50]:
pnl.sum()

2995.8052500000003

In [77]:
a = np.array([])
a = np.append(a,3)
a

array([3.])

In [74]:
a.append(3)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [160]:
def analysis(symbol):
    data = pd.read_csv("1hr/"+symbol+".csv")
    data['Date'] = pd.to_datetime(data['Date'])
    x = []
    for i in range(len(data)):
        if data.Date[i].hour>15:
            x.append(i)
    data = data.drop(x)
    data.reset_index(inplace = True, drop = True)
#     print(data.head(1015))
    lookback=14
    sma = 40
    data['ohlc'] = (data.High+data.Low+data.Close+data.Open)/4
    pnl=[]
    b = 0
    trade_position = []
    for i in range(15,len(data)):
#         trade_position.append(b)
        std = np.array(data.ohlc[i-lookback:i]).std()
        ohlc1,ohlc2,ohlc3 = data.ohlc[i-1],data.ohlc[i-2],data.ohlc[i-3]
        cond1 = ohlc1>ohlc2 and ohlc2>ohlc3
        cond2 = (ohlc1>ohlc2+0.5*std)
        cond3 = data.High[i-2]<data.Close[i-1]
      #  sig1 = np.sum(data.Close[i-sma:i])>np.sum(data.Close[i-sma-1:i-1]) and np.sum(data.Close[i-sma-1:i-1])>np.sum(data.Close[i-2-sma:i-2])
        sig2 = (((data.Close[i-1]-data.Open[i-lookback]))/(np.max(data.High[i-lookback:i])-np.min(data.Low[i-lookback:i])))>0.9
        lcond = (cond1 or cond2) and cond3 #and sig2
        if(lcond):
#             b = 1
            difference = (datetime(data['Date'][i].year, data['Date'][i].month, data['Date'][i].day, 15) - data['Date'][i]).total_seconds() / 3600
            
            p = []
#             print(int(difference + 1))
#             print(i)
            
            for k in range(int(difference + 1)):
#                 print(i + k)
                p.append(data.High[i + k])
            
            tp = max(p)
            
            if(tp > data.Open[i]*1.005):
                trade_position.append(0.005)
                pnl.append(0.005*data.Open[i])
            else:
                pnl.append(data.Close[i + difference]-data.Open[i])
                trade_position.append((data.Close[i + difference] - data.Open[i]) / data.Open[i])
        else:
            trade_position.append(0)
    
    mean = sum(trade_position) / len(trade_position)
    std = np.array(trade_position).std()
    sharpe = (mean / std) * math.sqrt(252)
    aa = np.array(pnl)
    return sharpe

In [161]:
analysis('RELIANCE')

-0.478977212393618

In [152]:
analysis

<function __main__.analysis(symbol)>

In [139]:
tickers = pd.read_csv('nifties/nifty500.csv').Symbol

In [24]:
df = []
for t in tickers:
    try:
        aa = analysis(t)
#         print(aa)
        df.append([t,aa.sum(),len(aa),len(aa[aa>0]),len(aa[aa<0])])
    except:
        print("error in ",t)

error in  360ONE
error in  AAVAS
error in  ATGL
error in  AWL
error in  ABSLAMC
error in  AETHER
error in  AFFLE
error in  ALOKINDS
error in  ANGELONE
error in  ANURAS
error in  APTUS
error in  BANDHANBNK
error in  BDL
error in  MAPMYINDIA
error in  CAMPUS
error in  CHALET
error in  CHEMPLASTS
error in  CLEAN
error in  CREDITACC
error in  DALBHARAT
error in  DELHIVERY
error in  DEVYANI
error in  EASEMYTRIP
error in  EQUITASBNK
error in  NYKAA
error in  FINEORG
error in  GRINFRA
error in  GLAND
error in  GOCOLORS
error in  GREENPANEL
error in  FLUOROCHEM
error in  HDFCAMC
error in  HAL
error in  POWERINDIA
error in  HOMEFIRST
error in  ISEC
error in  INDIAMART
error in  IRCTC
error in  IRFC
error in  INDIGOPNTS
error in  JUBLINGREA
error in  KPITTECH
error in  KALYANKJIL
error in  KIMS
error in  L&TFH
error in  LATENTVIEW
error in  LXCHEM
error in  LEMONTREE
error in  LICI
error in  MTARTECH
error in  LODHA
error in  M&MFIN
error in  M&M
error in  MAXHEALTH
error in  MAZDOCK
error in  M

In [25]:
df = pd.DataFrame(df,columns = ['Ticker','Totalpnl','trades','tradeswon','tradeslost'])

In [26]:
df['winratio'] = df.tradeswon/df.trades

In [27]:
len(df[df.winratio>0.7])

50

In [28]:
df[df.winratio>0.6].to_csv('For_aritra.csv')

In [38]:
df = []
for t in tickers:
    try:
        aa = analysis(t)
#         print(aa)
        df.append([t,aa])
    except:
        print("error in ",t)

error in  360ONE
error in  AAVAS
error in  ATGL
error in  AWL
error in  ABSLAMC
error in  AETHER
error in  AFFLE
error in  ALOKINDS
error in  ANGELONE
error in  ANURAS
error in  APTUS
error in  BANDHANBNK
error in  BDL
error in  MAPMYINDIA
error in  CAMPUS
error in  CHALET
error in  CHEMPLASTS
error in  CLEAN
error in  CREDITACC
error in  DALBHARAT
error in  DELHIVERY
error in  DEVYANI
error in  EASEMYTRIP
error in  EQUITASBNK
error in  NYKAA
error in  FINEORG
error in  GRINFRA
error in  GLAND
error in  GOCOLORS
error in  GREENPANEL
error in  FLUOROCHEM
error in  HDFCAMC
error in  HLEGLAS
error in  HAL
error in  POWERINDIA
error in  HOMEFIRST
error in  ISEC
error in  INDIAMART
error in  IRCTC
error in  IRFC
error in  INDIGOPNTS
error in  JUBLINGREA


/Users/raghavaggarwal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


error in  KPITTECH
error in  KALYANKJIL
error in  KIMS
error in  L&TFH
error in  LATENTVIEW
error in  LXCHEM
error in  LEMONTREE
error in  LICI
error in  MTARTECH
error in  LODHA
error in  M&MFIN
error in  M&M
error in  MAXHEALTH
error in  MAZDOCK
error in  MEDPLUS
error in  METROBRAND
error in  METROPOLIS
error in  MSUMI
error in  NAZARA
error in  NUVOCO
error in  PAYTM
error in  ORIENTELEC
error in  POLICYBZR
error in  PCBL
error in  PATANJALI
error in  POLYCAB
error in  POONAWALLA
error in  PRINCEPIPE
error in  RHIM
error in  RITES
error in  RVNL
error in  RAINBOW
error in  RTNINDIA
error in  RBA
error in  ROSSARI
error in  ROUTE
error in  SBICARD
error in  SAPPHIRE
error in  SHYAMMETL
error in  SONACOMS
error in  STARHEALTH
error in  SWSOLAR
error in  SUMICHEM
error in  SUVENPHAR
error in  TCNSBRANDS
error in  TATAMTRDVR
error in  UNOMINDA
error in  UTIAMC
error in  VARROC
error in  MANYAVAR
error in  VIJAYA
error in  ZFCVINDIA
error in  ZOMATO


In [40]:
#20may 20july
#4aug 16oct
#RELIANCE
df = pd.DataFrame(df,columns=['ticker','sharpe'])

In [43]:
df[df.sharpe>0]

,ticker,sharpe
3,AIAENG,0.201816
6,AARTIDRUGS,0.458281
7,ABBOTINDIA,0.220198
8,ADANIENT,0.392224
13,ABFRL,0.084654
...,...,...
377,UCOBANK,0.517685
381,UNIONBANK,0.404061
391,VINATIORGA,0.317449
396,WESTLIFE,0.232157


In [53]:
df[df.sharpe < 0]

,ticker,sharpe
0,3MINDIA,-0.105023
1,ABB,-0.140068
2,ACC,-0.009334
4,APLAPOLLO,-0.222660
5,AUBANK,-0.155947
...,...,...
401,ZEEL,-0.105702
402,ZENSARTECH,-0.221090
403,ZYDUSLIFE,-0.482735
404,ZYDUSWELL,-0.076765


In [129]:
data = pd.read_csv("1hr/TCS.csv")
data['Date'] = pd.to_datetime(data['Date'])


In [132]:

x

[1014, 2425, 4008, 4419, 4420]

In [134]:
data

,Date,Open,High,Low,Close,Volume
0,2018-03-01 10:00:00,1524.50,1524.93,1510.00,1511.00,105228
1,2018-03-01 11:00:00,1511.00,1520.10,1510.95,1519.62,42746
2,2018-03-01 12:00:00,1519.62,1520.03,1516.32,1519.85,27285
3,2018-03-01 13:00:00,1519.90,1525.00,1519.62,1524.97,58981
4,2018-03-01 14:00:00,1524.88,1530.07,1523.50,1523.50,86609
...,...,...,...,...,...,...
4434,2021-03-01 11:00:00,2951.75,2965.00,2946.70,2948.95,363940
4435,2021-03-01 12:00:00,2948.45,2948.95,2903.55,2916.85,545675
4436,2021-03-01 13:00:00,2917.95,2919.00,2901.80,2911.50,317690
4437,2021-03-01 14:00:00,2911.50,2923.20,2906.55,2907.05,363471
